# Wrapping a template library

A template library is a library where there are only template classes that can be instantiated.
Wrapping such libraries therefore requires **AutoWIG** to be able to consider various *C++* template classes instantiations during the `Parse` step.
The **Standard Template Library (STL)** library is a *C++* library that provides a set of common *C++* template classes such as containers and associative arrays.
These classes can be used with any built-in or user-defined type that supports some elementary operations (e.g. copying, assignment).
It is divided in four components called algorithms, containers, functional and iterators.
**STL** containers (e.g. `std::vector`, `std::set`) are used in many *C++* libraries.
In such a case, it does not seem relevant that every wrapped *C++* library contains wrappers for usual **STL** containers (e.g. `std::vector< double >`, `std::set< int >`).
We therefore proposed *Python* bindings for sequence containers (i.e. `pair`, `array`, `vector`, `deque`, `forward_list` and `list` of the `std` namespace) and associative containers (`set`, `multiset`, `map`, `multimap`, `unordered_set`, `unordered_multiset`, `unordered_map` and `unordered_multimap` of the `std` namespace).
These template instantiations are done for *C++* fundamental types (`bool`, `signed char`, `unsigned char`, `char`, `wchar_t`, `int` (with sign modifiers `signed` and `signed` combined or not with size modifiers `short`, `long` and `long long`), `float`, `double`, `long double`) and strings (`string`, `wstring` of the `std` namespace).
For ordered associative containers both `std::less` and `std::greater` comparators are used.

To wrap a template library, it is required to install the `pyclanglite` `parser`.

In [ ]:
%%bash
conda install -y -q python-clanglite -c statiskit

First, clone the **PySTL**  repository into the **PySTL** directory.

In [ ]:
%%bash
git clone https://github.com/StatisKit/PySTL.git

In the following we will use **SCons** software construction tool for compiling *C++* libraries.

In [ ]:
%%bash
conda install python-scons -c statiskit

As presented below, in order to wrap a template library, the user needs to write headers containing aliases for desired template class instantiations.

In [ ]:
%%bash
cd PySTL
python build.py
cd ..
pygmentize PySTL/src/cpp/vector.h

We import **AutoWIG** and create an empty Abstract Semantic Graph (ASG).

In [ ]:
import autowig
asg = autowig.AbstractSemanticGraph()

We need then to install the *C++* headers. This is done using the `cpp` target in **SCons**.

In [ ]:
%%bash
scons cpp -C PySTL

Once the headers habe been installed in the system, we parse headers with relevant compilation flags.

In [ ]:
from path import path
import sys
prefix = path(sys.prefix)
autowig.parser.plugin = 'pyclanglite'
asg = autowig.parser(asg,
                     [prefix/'include'/'pystl'/'vector.h'],
                     flags = ['-x', 'c++', '-std=c++11',
                              '-I' + str((prefix/'include').abspath())],
                     bootstrap = 1,
                     silent = True)

Since most of **AutoWIG** guidelines are respected, the `default` `controller` implementation is thus suitable.

In [ ]:
autowig.controller.plugin = 'default'
asg = autowig.controller(asg)

In [ ]:
autowig.generator.plugin = 'boost_python_internal'
wrappers = autowig.generator(asg,
                  module = './PySTL/src/py/_vector.cpp',
                  prefix = '_vector_wrapper_',
                  decorator = './PySTL/src/py/stl/vector/_vector.py',
                  closure = True)
wrappers.write()